In [1]:
# Import library
import os
import sys

# Gets directory where script was launched from
script_dir = os.getcwd()  
script_dir = script_dir + "/../lib/"
print(f"VSA Library: {script_dir}")

# Add the directory to Python's search path
sys.path.append(script_dir)  

import vsa
import numpy as np

VSA Library: /users/micas/rantonio/no_backup/kul_main/pytoy/vsa/../lib/


In [2]:
# Make codebooks and starting HVs
dimension = 1024

A = np.zeros((3,dimension))
B = np.zeros((3,dimension))
C = np.zeros((5,dimension))
D = np.zeros((5,dimension))

for i in range(3):
    A[i] = vsa.gen_hv(dimension, type='bipolar')
    B[i] = vsa.gen_hv(dimension, type='bipolar')

for i in range(5):
    C[i] = vsa.gen_hv(dimension, type='bipolar')
    D[i] = vsa.gen_hv(dimension, type='bipolar')

print(f'A.shape: {np.shape(A)}')
print(f'B.shape: {np.shape(B)}')
print(f'C.shape: {np.shape(C)}')
print(f'D.shape: {np.shape(D)}')
print(f'A sample: {A}')

A.shape: (3, 1024)
B.shape: (3, 1024)
C.shape: (5, 1024)
D.shape: (5, 1024)
A sample: [[-1.  1. -1. ...  1. -1.  1.]
 [ 1.  1.  1. ... -1. -1. -1.]
 [ 1.  1.  1. ... -1.  1.  1.]]


In [3]:
# Create a combination
bundled_hv_group = [
    vsa.hv_mult_list([A[0],B[0],C[3],D[4]]),
    vsa.hv_mult_list([A[1],B[1],C[2],D[2]]),
    vsa.hv_mult_list([A[1],B[1],C[3],D[3]]),
]

hv_bundled = np.array(vsa.hv_add(bundled_hv_group, sign_magnitude=True))

np.shape(hv_bundled)

(1024,)

The example above shows that the last 2 combinations have the same `A[1]` and `B[1]` but the `C` and `D` are different. How will the resonator network react?

In [4]:
# Estimators
num_iter = 50
a = np.zeros((num_iter,dimension))
b = np.zeros((num_iter,dimension))
c = np.zeros((num_iter,dimension))
d = np.zeros((num_iter,dimension))

# Initialize guesses
a[0] = np.sign(np.sum(A, axis=0))
b[0] = np.sign(np.sum(B, axis=0))
c[0] = np.sign(np.sum(C, axis=0))
d[0] = np.sign(np.sum(D, axis=0))

In [5]:
print(a[0])
print(np.sum(a[0]))

[ 1.  1.  1. ... -1. -1.  1.]
10.0


In [6]:
# Create query
hv_query = vsa.hv_mult(hv_bundled, A[1])

print(hv_query)

[-1.  1. -1. ...  1.  1.  1.]


In [7]:
# Iterate a few times only
for i in range(num_iter-1):
    b[i+1] = np.sign(np.matmul(B.T,np.matmul(B, np.array([vsa.hv_mult_list([hv_query, c[i], d[i]])]).T)).T)
    c[i+1] = np.sign(np.matmul(C.T,np.matmul(C, np.array([vsa.hv_mult_list([hv_query, b[i], d[i]])]).T)).T)
    d[i+1] = np.sign(np.matmul(D.T,np.matmul(D, np.array([vsa.hv_mult_list([hv_query, b[i], c[i]])]).T)).T)

In [8]:
for i in range(num_iter):
    print(f'{vsa.hv_cos(B[0],b[i])},{vsa.hv_cos(B[1],b[i])},{vsa.hv_cos(B[2],b[i])}')

0.529296875,0.45703125,0.46875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875

In [9]:
for i in range(num_iter):
    print(f'{vsa.hv_cos(C[0],c[i])},{vsa.hv_cos(C[1],c[i])},{vsa.hv_cos(C[2],c[i])},{vsa.hv_cos(C[3],c[i])},{vsa.hv_cos(C[4],c[i])}')

0.365234375,0.38671875,0.396484375,0.34765625,0.373046875
0.37109375,0.130859375,0.390625,0.603515625,0.1171875
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.025390625,-0.00390625,-0.005859375,1.0,-0.025390625
-0.02539

In [10]:
for i in range(num_iter):
    print(f'{vsa.hv_cos(D[0],d[i])},{vsa.hv_cos(D[1],d[i])},{vsa.hv_cos(D[2],d[i])},{vsa.hv_cos(D[3],d[i])},{vsa.hv_cos(D[4],d[i])}')

0.3359375,0.37109375,0.439453125,0.33203125,0.36328125
-0.283203125,0.193359375,0.28125,0.740234375,0.044921875
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.06640625,0.021484375,1.0,-0.00390625
-0.0234375,-0.0664062

The example above can either show it goes to `C[2]` and `D[2]` or `C[3]` and `D[3]`. So if you iterate it, it converges to at least one. We need to "take out" one of the decoded ones from bundled HV.

In [15]:
# Change this accordingly
take_away_bundle = vsa.hv_sub(vsa.hv_add(bundled_hv_group), vsa.hv_mult_list([A[1],B[1],C[3],D[3]]))
hv_take_away_bundle = np.sign(take_away_bundle)

In [16]:
hv_query = vsa.hv_mult(hv_take_away_bundle, A[1])
print(hv_query)

[-1.  1. -1. ...  1.  1. -0.]


In [17]:
# Estimators
num_iter = 50
a = np.zeros((num_iter,dimension))
b = np.zeros((num_iter,dimension))
c = np.zeros((num_iter,dimension))
d = np.zeros((num_iter,dimension))

# Initialize guesses
a[0] = np.sign(np.sum(A, axis=0))
b[0] = np.sign(np.sum(B, axis=0))
c[0] = np.sign(np.sum(C, axis=0))
d[0] = np.sign(np.sum(D, axis=0))

In [18]:
# Iterate a few times only
for i in range(num_iter-1):
    b[i+1] = np.sign(np.matmul(B.T,np.matmul(B, np.array([vsa.hv_mult_list([hv_query, c[i], d[i]])]).T)).T)
    c[i+1] = np.sign(np.matmul(C.T,np.matmul(C, np.array([vsa.hv_mult_list([hv_query, b[i], d[i]])]).T)).T)
    d[i+1] = np.sign(np.matmul(D.T,np.matmul(D, np.array([vsa.hv_mult_list([hv_query, b[i], c[i]])]).T)).T)

In [19]:
for i in range(num_iter):
    print(f'{vsa.hv_cos(B[0],b[i])},{vsa.hv_cos(B[1],b[i])},{vsa.hv_cos(B[2],b[i])}')

0.529296875,0.45703125,0.46875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875,1.0,-0.07421875
-0.013671875

In [20]:
for i in range(num_iter):
    print(f'{vsa.hv_cos(C[0],c[i])},{vsa.hv_cos(C[1],c[i])},{vsa.hv_cos(C[2],c[i])},{vsa.hv_cos(C[3],c[i])},{vsa.hv_cos(C[4],c[i])}')

0.365234375,0.38671875,0.396484375,0.34765625,0.373046875
0.134765625,0.15234375,0.865234375,0.12890625,0.119140625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.005859375,-0.015625
0.0,0.017578125,1.0,-0.0

In [21]:
for i in range(num_iter):
    print(f'{vsa.hv_cos(D[0],d[i])},{vsa.hv_cos(D[1],d[i])},{vsa.hv_cos(D[2],d[i])},{vsa.hv_cos(D[3],d[i])},{vsa.hv_cos(D[4],d[i])}')

0.3359375,0.37109375,0.439453125,0.33203125,0.36328125
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.009765625
-0.001953125,0.021484375,1.0,0.021484375,0.0

# Notes:

- In this version, after subtraction, we can still get the other combinations.
- I guess the only thing we have to fix is how to utilize the `stop` vectors.